In [13]:
"""[summary]
The script is to compute the cosine simliarities between queries and the document
The steps are as follows:
1. read the tf file
2. obtaint the input query
3. compute the cosine simliarities between query and each document
4.sort by the one with highest simliarity with the highest rank

"""

'[summary]\nThe script is to compute the cosine simliarities between queries and the document\nThe steps are as follows:\n1. read the tf file\n2. obtaint the input query\n3. compute the cosine simliarities between query and each document\n4.sort by the one with highest simliarity with the highest rank\n\n'

In [14]:
def readFile(_filePath):
    """[summary]

    Args:
        _filePath ([type]): [description]
    """
    import pandas as pd
    readFile = pd.read_csv(_filePath,index_col=0)
    return(readFile)

In [15]:
def queryVector(query,corpus):
    """[summary]
    vectorize query based on corpus
    
    Args:
        query ([list]): [list containing query terms]
    """
    corpusList = list(corpus.index)
    ## vector to track the corpusList
    vector = [0]*len(corpusList)
    ## format query\r\n",
    ## import scripts to process queries\r\n"
    import utils.query_processing
    ## convert to string if not string\r\n"
    if type(query) != str:
        query = ' '.join(query)
    ## clean the query
    query = utils.query_processing.process_query(query)

    "still lacking the query expansion"
    "Need  glove_kv and topn to be saved in the repo"
    # query = utils.query_processing.expand_query(query)    
    
    ## iterate over query terms
    for wordOfQuery in query:
    ## variable to tract the corpus index
        vectorIndex = 0
    ## iterate over the all corupus
        for wordOfCorpus in corpusList:
            if wordOfQuery ==  wordOfCorpus:
                vector[vectorIndex] += 1 
    ## update courpus index
            vectorIndex += 1               
    return(vector)

In [16]:
def cosineSimilarities(queryVector,docVector):
    "    \"\"\"[summary]\r\n",
    "    To compute the cosine similarity of the query with a single column/module in corpus\r\n",
    "    \r\n",
    "    Args:\r\n",
    "        queryVector ([list]): [query]\r\n",
    "        docVector ([list]): [description]\r\n"
    from scipy import spatial
    distance = 1 - spatial.distance.cosine(queryVector, docVector)
    return(round(distance,5))

In [17]:
def topModulesForEachQuery(_df):
    "[summary]\r\n",
    "    To obtain the top modules for each of the query\r\n",
    "\r\n",
    "    Args:\r\n",
    "        _df ([dataframe]): [description] data with rows of words and columns contain the modules, the cell of the dataframe is the similarity\r\n"
    overallTopModules = {}
    for query in _df.columns:
        singleModule = {}
        df.sort_values(by = query,ascending = False, inplace = True)
        ## obtain top 10 modules
        topModules = df[query][:10]
        singleModule['topModules'] = list(topModules.index)
        singleModule['topModulesScore'] = list(topModules)
        overallTopModules[str(query)] = (singleModule)
        
    import pandas as pd
    overallTopModulesDf = pd.DataFrame(overallTopModules)
    return(overallTopModulesDf)

In [20]:
if __name__ == "__main__":
    import pandas as pd
    corpus = readFile('../data/trained_scores/.csv') ## <-- assign the merge corupus here
    
    ## vaildation set
    """[summary]
    U can contorl which vaildation set to choose to test the cosine similarity model
    True : would be for the testingQuery that is self create which ensure a value when computing cosineSimilarities
    False : obtain the query based on 50% of vaildation dataset which might have all cosine similarities of zero
    """
    if False:
        testingQuery1 = ['ability','able','abstract']
        testingQuery2 = ['abstraction','accommodate','accompanies']
        testingQuery3 = ['account','accounting','achieve']
        testingQueryList = [[testingQuery1],[testingQuery2],[testingQuery3]]
        testingQueryDf = pd.DataFrame.from_records(testingQueryList,columns = ['querySample'])
        queries = testingQueryDf
        
    else:
        vaildationQueries = readFile('../data/survey/vaildation_sample_query.csv')
        queries = vaildationQueries
    
    ## iterate over the vaildationQueries
    queriesScore = {}
    for row,query in queries.T.iteritems():
        print('\nCurrent computing Query: {}'.format(query['querySample']))
        singleQueryVector = queryVector(query['querySample'],corpus)
        print("Number of terms in corpus: {}".format(sum(singleQueryVector)))
        
        CosineSimilaritiesScore = []
        for modules in list(corpus):
            SimilaritiesScore = cosineSimilarities(singleQueryVector,corpus[modules])
            CosineSimilaritiesScore.append(SimilaritiesScore)
            # print("Score for Query on {} : {}".format(modules,SimilaritiesScore))
        queriesScore[str(query['querySample'])] = CosineSimilaritiesScore
        
    ## Convert to dict to df
    modules = list(corpus)
    df  = pd.DataFrame.from_dict(queriesScore,orient='index',columns=modules)
    df = df.T
    
    ##obtaining top modules for each query
    rankedModule = topModulesForEachQuery(df)
    rankedModule



Current computing Query: ['track', 'large', 'perform', 'aws', 'automation']
Number of terms in corpus: 0

Current computing Query: ['secure', 'reinforcement', 'equally', 'simple', 'equilibrium']
Number of terms in corpus: 0

Current computing Query: ['dfs', 'track', 'equation', 'teammate', 'ec']
Number of terms in corpus: 0

Current computing Query: ['equilibrium', 'pi', 'monte', 'algorithm', 'server']
Number of terms in corpus: 0

Current computing Query: ['perform', 'decide', 'mapping', 'convolution', 'regression']
Number of terms in corpus: 0

Current computing Query: ['dimension', 'customize', 'strategic', 'wireframing', 'c']
Number of terms in corpus: 0

Current computing Query: ['tensorflow', 'interaction', 'work', 'devops', 'different']
Number of terms in corpus: 0

Current computing Query: ['dataloader', 'analysis', 'soft', 'crypto', 'tfidf']
Number of terms in corpus: 0

Current computing Query: ['ic', 'implementation', 'package', 'mysql', 'regression']
Number of terms in cor

C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\spatial\distance.py:728: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)



Current computing Query: ['simple', 'focus', 'decide', 'basic', 'strategic']
Number of terms in corpus: 0

Current computing Query: ['learnt', 'love', 'kera', 'wide', 'interaction']
Number of terms in corpus: 0

Current computing Query: ['darknet', 'node', 'hadoopmapreduce', 'reactjs', 'sklearn']
Number of terms in corpus: 0

Current computing Query: ['algorithm', 'secure', 'actually', 'teamwork', 'heuristic']
Number of terms in corpus: 0

Current computing Query: ['involve', 'screw', 'reverse', 'valuation', 'simple']
Number of terms in corpus: 0

Current computing Query: ['decide', 'excel', 'neural', 'tolerance', 'eqbm']
Number of terms in corpus: 0

Current computing Query: ['lot', 'visualization', 'prototyping', 'document', 'player']
Number of terms in corpus: 0

Current computing Query: ['application', 'people', 'monte', 'cloud', 'actually']
Number of terms in corpus: 0

Current computing Query: ['rank', 'optimum', 'class', 'mongodb', 'scholes']
Number of terms in corpus: 0

Curre

In [21]:
rankedModule

,"['track', 'large', 'perform', 'aws', 'automation']","['secure', 'reinforcement', 'equally', 'simple', 'equilibrium']","['dfs', 'track', 'equation', 'teammate', 'ec']","['equilibrium', 'pi', 'monte', 'algorithm', 'server']","['perform', 'decide', 'mapping', 'convolution', 'regression']","['dimension', 'customize', 'strategic', 'wireframing', 'c']","['tensorflow', 'interaction', 'work', 'devops', 'different']","['dataloader', 'analysis', 'soft', 'crypto', 'tfidf']","['ic', 'implementation', 'package', 'mysql', 'regression']","['hedge', 'statement', 'day', 'reactjs', 'real']",...,"['algorithm', 'secure', 'actually', 'teamwork', 'heuristic']","['involve', 'screw', 'reverse', 'valuation', 'simple']","['decide', 'excel', 'neural', 'tolerance', 'eqbm']","['lot', 'visualization', 'prototyping', 'document', 'player']","['application', 'people', 'monte', 'cloud', 'actually']","['rank', 'optimum', 'class', 'mongodb', 'scholes']","['mongo', 'operation', 'real', 'crypto', 'figma']","['work', 'esp', 'pyspark', 'tensorflow', 'cod']","['ec', 'software', 'winnable', 'remember', 'database']","['reverse', 'share', 'teammate', 'type', 'interaction']"
topModules,[50.006 User Interface Design and Implementati...,[50.006 User Interface Design and Implementati...,[50.006 User Interface Design and Implementati...,[50.006 User Interface Design and Implementati...,[50.006 User Interface Design and Implementati...,[50.006 User Interface Design and Implementati...,[50.006 User Interface Design and Implementati...,[50.006 User Interface Design and Implementati...,[50.006 User Interface Design and Implementati...,[50.006 User Interface Design and Implementati...,...,[50.006 User Interface Design and Implementati...,[50.006 User Interface Design and Implementati...,[50.006 User Interface Design and Implementati...,[50.006 User Interface Design and Implementati...,[50.006 User Interface Design and Implementati...,[50.006 User Interface Design and Implementati...,[50.006 User Interface Design and Implementati...,[50.006 User Interface Design and Implementati...,[50.006 User Interface Design and Implementati...,[50.006 User Interface Design and Implementati...
topModulesScore,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
